# 1. Set up the Huggingface Climate Policy Radar dataset.

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset

ds = load_dataset("ClimatePolicyRadar/all-document-text-data")

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

In [10]:
chunks = ds["train"]

In [11]:
chunks[0]

{'document_id': 'CCLW.document.i00000002.n0000',
 'document_metadata.collection_summary': None,
 'document_metadata.collection_title': None,
 'document_metadata.corpus_type_name': 'Laws and Policies',
 'document_metadata.corpus_import_id': 'CCLW.corpus.i00000001.n0000',
 'document_metadata.category': 'Executive',
 'document_metadata.description': '<p><span style="font-size: 10pt;font-family: Arial;">This plan aims to address the five dimensions of the Energy Union, namely energy security, the internal energy market, energy efficiency, decarbonization, and research, innovation, and competitiveness. It seeks to review targets and policies in the energy sector to alleviate energy poverty and provide affordable energy, to provide and ensure accessibility to quantitative data on the sector as well as to inform relevant stakeholders on the plan. The key policies and objectives focus on the increase in renewable energy sources, reduction of fossil fuels in transport and industry, incorporatio

In [13]:
chunks[:10]["text_block.text"]

['Draft of the National Energy and Climate Plan of the Republic of Albania',
 'July 2021',
 'REPUBLIKA SHOIPERISE',
 'MINISTRIA E TURIZMIT DHE MJEDISIT',
 'MINISTRIA E INFRASTRUKTURĒS DHE ENERGJISE',
 'german cooperation DEUTSCHE ZUSAMMENARBEIT',
 'Implemented by giz',
 'Deutsche Gesellschaft Für Internationale Zusammenarbeit (312) CmbH',
 'Responsible for this document: Ministry of Infrastructure and Energy of Albania',
 'Purpose of this document: Submission to Energy Community for comments, in order to improve the final version of the NECP document']